Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Model deployment

Please ensure you have run all previous notebooks in sequence before running this.

Please Register Azure Container Instance(ACI) using Azure Portal: https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-manager-supported-services#portal in your subscription before using the SDK to deploy your ML model to ACI.

In [4]:
from azureml.core import Workspace
import azureml.core
import os

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

config_path = '/dbfs/tmp/'

#'''
ws = Workspace.from_config(path=os.path.join(config_path, 'aml_config', 'config.json'))
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')
#'''

In [5]:
## NOTE: service deployment always gets the model from the current working dir.
model_name = "PdM_logistic_regression.mml" # 
model_name_dbfs = os.path.join("/dbfs", model_name)

print("copy model from dbfs to local")
model_local = "file:" + os.getcwd() + "/" + model_name
dbutils.fs.cp(model_name, model_local, True)

In [6]:
# register the model
from azureml.core.model import Model
mymodel = Model.register(model_path = model_name, # this points to a local file
                       model_name = model_name, # this is the name the model is registered as, am using same name for both path and name.                 
                       description = "ADB trained model by an amazing data scientist",
                       workspace = ws)

print(mymodel.name, mymodel.description, mymodel.version)

In [7]:
#%%writefile score_sparkml.py
score_sparkml = """

import json

def init():
    # One-time initialization of PySpark and predictive model
    import pyspark
    from azureml.core.model import Model
    from pyspark.ml import PipelineModel

    global trainedModel
    global spark

    spark = pyspark.sql.SparkSession.builder.appName("ADB and AML notebook by an amazing data scientist").getOrCreate()
    model_name = "{model_name}" #interpolated
    model_path = Model.get_model_path(model_name)
    trainedModel = PipelineModel.load(model_path)
    
def run(input_json):
    if isinstance(trainedModel, Exception):
        return json.dumps({{"trainedModel":str(trainedModel)}})
      
    try:
        sc = spark.sparkContext
        input_list = json.loads(input_json)
        input_rdd = sc.parallelize(input_list)
        input_df = spark.read.json(input_rdd)
    
        # Compute prediction
        prediction = trainedModel.transform(input_df)
        #result = prediction.first().prediction
        predictions = prediction.collect()

        #Get each scored result
        preds = [str(x['prediction']) for x in predictions]
        result = ",".join(preds)
        # you can return any data type as long as it is JSON-serializable
        return json.dumps({{"result":result}})        
    except Exception as e:
        result = str(e)
        return json.dumps({{"error":result}})
    
""".format(model_name=model_name)

exec(score_sparkml)

with open("score_sparkml.py", "w") as file:
    file.write(score_sparkml)

In [8]:
from azureml.core.conda_dependencies import CondaDependencies 

myacienv = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas']) #showing how to add libs as an example - not needed for this model.

with open("mydeployenv.yml","w") as f:
    f.write(myacienv.serialize_to_string())

In [9]:
# deploy to ACI
from azureml.core.webservice import AciWebservice, Webservice

myaci_config = AciWebservice.deploy_configuration(
    cpu_cores = 2, 
    memory_gb = 2, 
    tags = {'name':'Databricks Azure ML ACI'}, 
    description = 'This is for ADB and AML example. Azure Databricks & Azure ML SDK demo with ACI.')

In [10]:
# this will take 10-15 minutes to finish

service_name = "aciws"
runtime = "spark-py" 
driver_file = "score_sparkml.py"
my_conda_file = "mydeployenv.yml"

# image creation
from azureml.core.image import ContainerImage
myimage_config = ContainerImage.image_configuration(execution_script = driver_file, 
                                    runtime = runtime, 
                                    conda_file = my_conda_file)

# Webservice creation
myservice = Webservice.deploy_from_model(
  workspace=ws, 
  name=service_name,
  deployment_config = myaci_config,
  models = [mymodel],
  image_config = myimage_config)

myservice.wait_for_deployment(show_output=True)

In [11]:
# List images by ws

for i in ContainerImage.list(workspace = ws):
    print('{}(v.{} [{}]) stored at {} with build log {}'.format(i.name, i.version, i.creation_state, i.image_location, i.image_build_log_uri))

In [12]:
#for using the Web HTTP API 
print(myservice.scoring_uri)

In [13]:
%sh

ls -la /dbfs/PdM_logistic_regression/*

In [14]:
import json
from pyspark.sql.types import *

test_data_path = "PdM_logistic_regression"
df_test = spark.read.parquet(test_data_path).limit(5)
test_json = json.dumps(df_test.toJSON().collect())

print(test_json)

In [15]:
myservice.run(input_data = test_json)

In [16]:
#comment below line to not delete the web service
myservice.delete()

Please make sure to install **VS Code** on your compute *before* the next session tmrw morning.

You can find binary installers for VS Code here:

[https://code.visualstudio.com/download](https://code.visualstudio.com/download)

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.